In [105]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',500)

In [112]:
df_admi = pd.read_csv('./dataset/ADMISSIONS.csv')
df_diag = pd.read_csv('./dataset/DIAGNOSES_ICD.csv')
df_los = pd.read_csv('./dataset/ICUSTAYS.csv')
df_patient = pd.read_csv('./dataset/PATIENTS.csv')
df_ditems = pd.read_csv('./dataset/D_ITEMS.csv')

df_admi = df_admi.dropna()
df_admi = df_admi.drop_duplicates(subset=['SUBJECT_ID'],keep = 'first')
df_admi.drop([
    "ROW_ID",
    "ADMITTIME",
    "DISCHTIME",
    "DEATHTIME",
    "EDREGTIME",
    "EDOUTTIME",
    "HOSPITAL_EXPIRE_FLAG",
    "HAS_CHARTEVENTS_DATA",
    "DIAGNOSIS",
    "LANGUAGE",
    "HADM_ID"
], axis=1, inplace=True)

# df_diag = df_diag.drop_duplicates(subset=['SUBJECT_ID','HADM_ID'],keep = 'first')
df_diag = df_diag.dropna()
df_diag.drop([
    "ROW_ID",
    "SEQ_NUM",
    "HADM_ID"
],axis = 1, inplace = True)
# df_diag = df_diag.loc[df_diag["ICD9_CODE"].str.contains('99591')]
df_diag.loc[df_diag["ICD9_CODE"].str.contains('99591'),"ICD9_CODE"] = '\+1'
df_diag.loc[~df_diag["ICD9_CODE"].str.contains('\+1'),"ICD9_CODE"]= '\-1' 
# df_diag = df_diag.loc[df_diag["ICD9_CODE"] != "99591","ICD9_CODE"] = '0'
# df_diag = df_diag.loc[df_diag["ICD9_CODE"].str.contains('\-1')]
def label_sepsis (e):
    if e in '\+1':
        return 1
    else:
        return 0
df_diag['is_sepsis'] = df_diag['ICD9_CODE'].apply(lambda e :label_sepsis(e))
df_diag.drop([
    "ICD9_CODE",
],axis = 1, inplace = True)
df_diag = df_diag.sort_values('is_sepsis')
df_diag = df_diag.drop_duplicates(subset = 'SUBJECT_ID', keep = "last")
# df_diag = df_diag.loc[df_diag['is_sepsis'] == 0]

# df_los = df_los.drop_duplicates(subset=['SUBJECT_ID'],keep = 'last')
df_los = df_los.dropna()
df_los = df_los.groupby('SUBJECT_ID').mean().reset_index()
df_los.drop([
    "ROW_ID",
    "ICUSTAY_ID",
    "FIRST_WARDID",
    "LAST_WARDID",
    "HADM_ID"
],axis = 1, inplace = True)
df_los.LOS = df_los.LOS.astype(int)
df_los = df_los

# df_patient = df_patient.drop_duplicates(subset=['SUBJECT_ID','HADM_ID'],keep = 'first')
df_patient = df_patient.dropna()
df_patient['DOB'] = pd.to_datetime(df_patient['DOB']).dt.date
df_patient['DOD'] = pd.to_datetime(df_patient['DOD']).dt.date
df_patient['age'] = df_patient.apply(lambda e:(e['DOD'] - e['DOB']).days/365,axis = 1)
df_patient.age = df_patient.age.astype(int)
df_patient.drop([
    "ROW_ID",
    "EXPIRE_FLAG",
    "DOD_SSN",
    "DOD_HOSP",
    "DOD",
    "DOB"
],axis = 1, inplace = True)
df_patient = df_patient.loc[df_patient['age']<=100]

# df_admi
# df_diag
df_los
# df_patient

# df_merge = pd.merge(df_admi, df_diag, on="SUBJECT_ID")
# df_merge = pd.merge(df_merge,df_los, on="SUBJECT_ID")
# df_merge = pd.merge(df_merge,df_patient, on="SUBJECT_ID")
# df_merge

,SUBJECT_ID,LOS
0,2,0
18361,19481,0
18362,19482,0
18366,19486,0
18368,19488,0
...,...,...
2847,3014,150
6586,6975,151
15330,16244,167
28083,29899,169
